In [0]:
import pyspark.sql.functions as F
import os

STORAGE_ACCOUNT = os.getenv('STORAGE_ACCOUNT')
STORAGE_ACCOUNT_KEY = os.getenv('STORAGE_ACCOUNT_KEY')
spark.conf.set(STORAGE_ACCOUNT, STORAGE_ACCOUNT_KEY)


In [0]:
pacific_df = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("abfss://datasets@tfmstorageacc.dfs.core.windows.net/violence_against_women_pacific.csv")

pacific_df.limit(20).display()

STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,FREQ,Frequency,TIME_PERIOD,Time,GEO_PICT,Pacific Island Countries and territories,TOPIC10,Topic11,INDICATOR12,Indicator13,SEX14,Sex15,AGE16,Age17,CONDITION,Women’s condition,VIOLENCE_TYPE,Type of violence,PERPETRATOR22,Perpetrator23,ACTUALITY24,Actuality25,LIFEPER,Period of life,OUTCOME28,Outcome29,RESPONSE30,Response31,HELP_REASON,Reason for searching help,HELP_PROVIDER,Help provider,OBS_VALUE,Observation value,UNIT_MEASURE,Unit of measure,UNIT_MULT,Unit multiplier,OBS_STATUS,Observation Status,DATA_SOURCE,Data source,OBS_COMMENT,Comment
DATAFLOW,SPC:DF_VAW(1.0),Violence against women,I,A,Annual,2006,null,WS,Samoa,VAW_TOPIC_005,Acts of emotional violence by partners,NUMPERRF,Number of persons in relative frequency,F,Female,Y15T64,15-64,EVPART,Ever-partnered,EMO_SCA,Scared or intimidated,PARTNER,Partner,ALOLIFE,At least once in their lifetime,_T,Any,_T,Any,_T,Any,_T,Any,_T,Any,9.9,null,PERCENT,percent,null,null,null,null,null,null,Women 15-49.,null
DATAFLOW,SPC:DF_VAW(1.0),Violence against women,I,A,Annual,2006,null,WS,Samoa,VAW_TOPIC_012,Impact of partner violence on women's health and wellbeing,NUMPERRF,Number of persons in relative frequency,F,Female,Y15T64,15-64,_T,Any,_T,Any,PARTNER,Partner,_T,Any,_T,Any,SUICIDEATT,Ever attempting suicide,_T,Any,_T,Any,_T,Any,5.3,null,PERCENT,percent,null,null,null,null,null,null,Women 15-49.,null
DATAFLOW,SPC:DF_VAW(1.0),Violence against women,I,A,Annual,2006,null,WS,Samoa,VAW_TOPIC_017,Seeking and receiving help,NUMPERRF,Number of persons in relative frequency,F,Female,Y15T64,15-64,_T,Any,_T,Any,_T,Any,_T,Any,_T,Any,_T,Any,_T,Any,_T,Any,FRIENDS,Friends,9.2,null,PERCENT,percent,null,null,null,null,null,null,Women 15-49.,null
DATAFLOW,SPC:DF_VAW(1.0),Violence against women,I,A,Annual,2007,null,TV,Tuvalu,VAW_TOPIC_006,Acts of controlling behaviours by partners,NUMPERRF,Number of persons in relative frequency,F,Female,Y15T64,15-64,EVPART,Ever-partnered,CONT_UNFAITH,Partner often suspicious she is unfaithful,PARTNER,Partner,ALO12M,At least once in the past 12 months,_T,Any,_T,Any,_T,Any,_T,Any,_T,Any,null,null,PERCENT,percent,null,null,O,Missing value,null,null,Ever married women 15-49.,null
DATAFLOW,SPC:DF_VAW(1.0),Violence against women,I,A,Annual,2007,null,TV,Tuvalu,VAW_TOPIC_007,Types of violence against women by others (non-partners),NUMPERRF,Number of persons in relative frequency,F,Female,Y15T64,15-64,_T,Any,SEX,Sexual violence,NONPART,Non-partner,ALOLIFE,At least once in their lifetime,SINCE15,Since the age of 15,_T,Any,_T,Any,_T,Any,_T,Any,null,null,PERCENT,percent,null,null,O,Missing value,null,null,Ever married women 15-49.,null
DATAFLOW,SPC:DF_VAW(1.0),Violence against women,I,A,Annual,2007,null,TV,Tuvalu,VAW_TOPIC_013,Impact of partner violence on women's reproductive health,NUMPERRF,Number of persons in relative frequency,F,Female,Y15T64,15-64,EVPREG,Ever pregnant,_T,Any,PARTNER,Partner,_T,Any,_T,Any,STILB,Ever had stillbirths,_T,Any,_T,Any,_T,Any,null,null,PERCENT,percent,null,null,O,Missing value,null,null,Ever married women 15-49.,null
DATAFLOW,SPC:DF_VAW(1.0),Violence against women,I,A,Annual,2007,null,TV,Tuvalu,VAW_TOPIC_015,Impact of partner violence on the wellbeing of children,NUMPERRF,Number of persons in relative frequency,F,Female,Y15T64,15-64,CHI614,With children 6-14 years old,_T,Any,_T,Any,_T,Any,_T,Any,AL1CHILDQUIET,At least one of her children being quiet/withdrawn,_T,Any,_T,Any,_T,Any,null,null,PERCENT,percent,null,null,O,Missing value,null,null,Ever married women 15-49.,null
DATAFLOW,SPC:DF_VAW(1.0),Violence against women,I,A,Annual,2007,null,TV,Tuvalu,VAW_TOPIC_018,Reasons for seeking help,NUMPERRF,Number of persons in relative frequency,F,Female,Y15T64,15-64,_T,Any,_T,Any,PARTNER,Partner,_T,Any,_T,Any,_T,Any,_T,Any,THRKILL,Threatened or tried to kill her,_T,Any,null,null,PERCENT,percent,null,null,O,Missing value,null,null,Ever married women 15-49.,null
DATAFLOW,SPC:DF_VAW(1.0),Violence against women,I,A,Annual,200

There are a lot of columns and statistics, the first thing we need to do is select the statistics we are interested on, so lets see which are avaliable

In [0]:
pacific_df.select(
    F.col('TOPIC10'),
    F.col('Topic11'),
).distinct().limit(20).display()

TOPIC10,Topic11
VAW_TOPIC_012,Impact of partner violence on women's health and wellbeing
VAW_TOPIC_006,Acts of controlling behaviours by partners
VAW_TOPIC_014,Impact of partner violence on women's work who work for money
VAW_TOPIC_001,Types of violence against women by partner
VAW_TOPIC_008,Non-Partner Physical violence by type of perpetrator
VAW_TOPIC_016,Responses to partner violence - women told others about violence or leave home
VAW_TOPIC_011,Injuries from physical or sexual partner violence
VAW_TOPIC_002,Partner Physical violence by severity
VAW_TOPIC_005,Acts of emotional violence by partners
VAW_TOPIC_007,Types of violence against women by others (non-partners)


The topics we are interested in are 
- VAW_TOPIC_001: Types of violence against women by partner
- VAW_TOPIC_007: Types of violence against women by others (non-partners)
- VAW_TOPIC_010: Child sexual abuse prevalence by type of perpetrator
We don't want null values.

In [0]:
pacific_df = pacific_df.where(
    F.col('TOPIC10').isin('VAW_TOPIC_001', 'VAW_TOPIC_007', 'VAW_TOPIC_010') &
    F.col('OBS_VALUE').isNotNull()
).select(
    F.col('TIME_PERIOD').alias('year'),
    F.col('Pacific Island Countries and territories').alias('country'),
    F.col('Type of violence').alias('violence_type'),
    F.col('OBS_VALUE').alias('value_perc'),
    F.col('Perpetrator23').alias('perpetrator'),
).orderBy(
    F.col('year').desc(),
    F.col('country'),
)
pacific_df.limit(20).display()

year,country,violence_type,value_perc,perpetrator
2019,Kiribati,Sexual violence,23.7,Partner
2019,Kiribati,Emotional violence,38.5,Partner
2019,Kiribati,Physical and/or sexual violence,54.0,Partner
2019,Kiribati,Physical violence,39.3,Partner
2019,Kiribati,Physical violence,7.8,Non-partner
2019,Kiribati,Sexual violence,5.1,Non-partner
2019,Kiribati,Sexual violence,10.3,Non-partner
2019,Kiribati,Any,13.5,Any
2019,Kiribati,Physical violence,18.9,Non-partner
2019,Kiribati,Sexual violence,21.4,Partner


There are some repeated fields with just the value differing, that is due to some fields outside the selection that make it so. In order to delete those we will get the higher percentage as that is the best we can do to reduce the number of cases lost. 

In [0]:
pacific_df = pacific_df.groupBy('year', 'country', 'violence_type', 'perpetrator').agg(
    F.max(F.col('value_perc')).alias('value_perc')
)
pacific_df.limit(20).display()

year,country,violence_type,perpetrator,value_perc
2013,Cook Islands,Physical violence,Partner,30.2
2009,Tonga,Physical violence,Non-partner,67.8
2011,Fiji,Child sexual abuse,Any,15.6
2011,Fiji,At least one act of economic abusive,Partner,28.3
2019,Kiribati,Physical violence,Partner,50.9
2014,Micronesia (Federated States of),Child sexual abuse,Any,14.2
2012,Marshall Islands,Sexual violence,Non-partner,13.0
2019,Tonga,Physical and/or sexual violence,Partner,19.1
2013,Cook Islands,Physical violence,Non-partner,38.6
2013,Nauru,Physical violence,Partner,46.6
